In [22]:
import os

import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

# Download data and set working directory

os.chdir(os.path.join(et.io.HOME, 'data'))

In [23]:
odm_data_path = os.path.join("odm_orthophoto.tif")

odm_data = rxr.open_rasterio(odm_data_path)

# View shape of the data
odm_data.shape

(5, 6269, 5977)

In [40]:
coord_names = ['x', 'y']
odm_data.drop_indexes(coord_names,*, errors='raise')

SyntaxError: iterable argument unpacking follows keyword argument unpacking (2267856661.py, line 2)

In [28]:
odm_data[3]

<xarray.DataArray (y: 6269, x: 5977)>
[37469813 values with dtype=uint16]
Coordinates:
    band         int64 4
  * x            (x) float64 5.6e+05 5.6e+05 5.6e+05 ... 5.603e+05 5.603e+05
  * y            (y) float64 5.689e+06 5.689e+06 ... 5.689e+06 5.689e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        255
    STATISTICS_MEAN:           75.384230696838
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         68.290659271143
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0
    long_name:                 ('Green', 'Red', 'RedEdge', 'NIR', None)

In [26]:
odm_ndvi = es.normalized_diff(odm_data[2], odm_data[0])

/opt/conda/lib/python3.11/site-packages/earthpy/spatial.py:109: Warning: Divide by zero produced infinity values that will be replaced with nan values
  warnings.warn(


IndexError: 2-dimensional boolean indexing is not supported. 

In [7]:
ep.plot_bands(odm_ndvi,
              cmap='PiYG',
              scale=False,
              vmin=-1, vmax=1,
              title="NDVI Am Sande 15. September 2023")
plt.show()

NameError: name 'odm_ndvi' is not defined

In [94]:
# A script to calculate the NDVI from a color-infrared orthophoto.
# requires python-gdal

import numpy
import argparse
import os.path
try:
    from osgeo import gdal
    from osgeo import osr
except ImportError:
    raise ImportError("You need to install python-gdal. run `apt-get install python-gdal`")
    exit()


def parse_args():
    p = argparse.ArgumentParser("A script that calculates the NDVI of a CIR orthophoto")

    p.add_argument("orthophoto", metavar="/home/jovyan/data/odm_orthophoto.original.tif",
                   type=argparse.FileType('r'),
                   help="The CIR orthophoto. Must be a GeoTiff.")
    p.add_argument("4", metavar= "NIR", type=int,
                   help="NIR band number")
    p.add_argument("2", metavar= "VIS", type=int,
                   help="Vis band number")
   # p.add_argument("out", metavar="/home/jovyan/results/ndvi.tif>",
   #                type=argparse.FileType('w'),
   #                help="The output file. Also must be in GeoTiff format")
   # p.add_argument("--overwrite", "-o",
   #                action='store_true',
   #                default=False,
   #                help="Will overwrite output file if it exists. ")
    return p.parse_args()


def calc_ndvi(nir, vis):
    """
    Calculates the NDVI of an orthophoto using nir and vis bands.
    :param nir: An array containing the nir band
    :param vis: An array containing the vis band
    :return: An array that will be exported as a tif
    """

    # Take the orthophoto and do nir - vis / nir + vis
    # for each cell, calculate ndvi (masking out where divide by 0)
    ndvi = numpy.empty(nir.shape, dtype=float)
    mask = numpy.not_equal((nirb + visb), 0.0)
    return numpy.choose(mask, (-1.0, numpy.true_divide(numpy.subtract(nirb, visb), numpy.add(nirb, visb))))




In [95]:
args = parse_args()

usage: A script that calculates the NDVI of a CIR orthophoto
       [-h] /home/jovyan/data/odm_orthophoto.original.tif NIR VIS
A script that calculates the NDVI of a CIR orthophoto: error: the following arguments are required: NIR, VIS


SystemExit: 2

In [60]:
#if __name__ == "__main__":

rootdir = os.path.dirname(os.path.abspath('/home/jovyan/data/odm_orthophoto.tif'))

# Parse args
args = parse_args()

if not args.overwrite and os.path.isfile(os.path.join(rootdir, args.out.name)):
    print("File exists, rename or use -o to overwrite.")
    exit()
# import raster
raster = gdal.Open(args.orthophoto.odm_orthophoto.tif)
orthophoto = raster.ReadAsArray()
# parse out bands
nirb = orthophoto[args.nir - 1].astype(float)
visb = orthophoto[args.vis - 1].astype(float)

outfile = args.out

# Do ndvi calc
ndvi = calc_ndvi(nirb, visb)

# export raster
out_driver = gdal.GetDriverByName('GTiff')\
    .Create(outfile.name, int(ndvi.shape[1]), int(ndvi.shape[0]), 1, gdal.GDT_Float32)
outband = out_driver.GetRasterBand(1)
outband.WriteArray(ndvi)
outcrs = osr.SpatialReference()
outcrs.ImportFromWkt(raster.GetProjectionRef())
out_driver.SetProjection(outcrs.ExportToWkt())
out_driver.SetGeoTransform(raster.GetGeoTransform())
outband.FlushCache()

usage: A script that calculates the NDVI of a CIR orthophoto
       [-h] [--overwrite] odm_orthophoto.tif 4 2 ./results/ndvi.tif>
A script that calculates the NDVI of a CIR orthophoto: error: the following arguments are required: 4, 2, ./results/ndvi.tif>


SystemExit: 2

In [58]:
args

NameError: name 'args' is not defined